# Purge Entities

This script can be used to delete test entities in an epilot organization.

Warning: Only run this notebook if you are ok with losing data! Should not be used in production organisations.

In [ ]:
import entityClient from 'epilot-sdk/entity-client'
import { authorizeWithToken } from 'epilot-sdk/auth'

authorizeWithToken(entityClient, `<your token>`)

entityClient.api.initialized

In [ ]:
import PQueue from 'p-queue'

// deletes a batch of 50 entities in parallel
const deleteEntities = async (items: { _id: string, _schema: string }[]) => {
  const jobs = new PQueue({ concurrency: 50 })
  
  for (const item of items) {
    jobs.add(async () => {
      try {
        const deleteRes: any = await entityClient.deleteEntity({ slug: item._schema, id: item._id })
        // console.info('Deleted entity', item._schema, deleteRes.data._title, item._id)
      } catch (err) {
        console.error('Could not delete entity', item)
      }
    })
  }
  
  await jobs.onIdle();
}

In [ ]:
const BATCH_SIZE = 250

// define your search query here. these entities will be deleted.
let searchRes = await entityClient.searchEntities(null, {
  q: '_schema:contact _schema:contract _schema:meter',
  fields: ['_schema', '_id', '_org'],
  size: BATCH_SIZE,
  from: 0,
})

searchRes.data.hits

In [ ]:
// delete entities in batches
while (searchRes.data.hits > 0) {
  console.log('Remaining: ', searchRes.data.hits)
  
  const [nextSearchRes] = await Promise.all([
    // fetch next batch
    entityClient.searchEntities(null, {
      q: '_schema:contact _schema:contract _schema:meter',
      fields: ['_schema', '_id', '_org'],
      size: BATCH_SIZE,
      from: BATCH_SIZE,
    }),
    // delete current batch
    deleteEntities(searchRes.data.results)
  ])
  
  searchRes = nextSearchRes
} 